In [20]:
import pandas as pd
import numpy as np

In [21]:
companies = pd.read_csv("/home/musasina/projects/teknofest/msnet/datasets/companies.csv").dropna()

In [22]:
companies = companies.convert_dtypes(convert_string=True)

In [23]:
companies_array = np.array(companies).flatten()

In [24]:
df_train = pd.read_csv("/home/musasina/projects/teknofest/msnet/datasets/train_turkish.csv")

In [25]:
def cut_text(text:str):
    splitted = str(text).split()
    splitted = splitted[:15]
    return " ".join(splitted)

In [26]:
df_train["text"] = df_train["text"].map(cut_text)

In [27]:
df_train_texts = df_train["text"].to_list()

In [28]:
labels = {"Notr":0,"Positive":2,"Negative":1}

In [29]:
sentiment = df_train["label"].map(str).to_numpy()

In [30]:
sentiment[0]

'Notr'

In [31]:
sentiment = np.apply_along_axis(lambda x : labels[x[0]],axis=1,arr=sentiment.reshape((-1,1)))

In [32]:
notr_bow = pd.read_csv("/home/musasina/projects/teknofest/msnet/bows/notr_bow.csv")
positive_bow = pd.read_csv("/home/musasina/projects/teknofest/msnet/bows/positive_bow.csv")
negative_bow = pd.read_csv("/home/musasina/projects/teknofest/msnet/bows/negative_bow.csv")

In [33]:
notr_bow_set = set(notr_bow["words"].map(str).to_list())
positive_bow_set = set(positive_bow["words"].map(str).to_list())
negative_bow_set = set(negative_bow["words"].map(str).to_list())

In [34]:
wrong_bow_set = notr_bow_set.intersection(positive_bow_set,negative_bow)
wrong_bow_set_pos_notr = notr_bow_set.intersection(positive_bow_set)
wrong_bow_set_pos_neg = negative_bow_set.intersection(positive_bow_set)
wrong_bow_set_neg_notr = negative_bow_set.intersection(notr_bow_set)

In [35]:
print(len(wrong_bow_set_pos_notr))
print(len(wrong_bow_set_pos_neg))
print(len(wrong_bow_set_neg_notr))

9488
21008
11867


In [36]:
notr_bow_set -= wrong_bow_set
positive_bow_set -= wrong_bow_set
negative_bow_set -= wrong_bow_set
#----------------------------------
notr_bow_set -= wrong_bow_set_pos_notr
positive_bow_set -= wrong_bow_set_pos_notr
#----------------------------------
positive_bow_set -= wrong_bow_set_pos_neg
negative_bow_set -= wrong_bow_set_pos_neg
#----------------------------------
notr_bow_set -= wrong_bow_set_neg_notr
negative_bow_set -= wrong_bow_set_neg_notr

In [37]:
texts_with_companies = []

for text,label in zip(df_train_texts,sentiment):
    for _ in range(1):
        company = np.random.choice(companies_array)
        splitted_text = text.split()
        labels = [0]*15
        index = np.random.randint(0,len(splitted_text))
        for i,word in enumerate(splitted_text):
            if word in notr_bow_set:
                labels[i]=0
            elif word in positive_bow_set:
                labels[i]=2
            elif word in negative_bow_set:
                labels[i]=1
        labels.insert(index,label)
        splitted_text.insert(index,company)
        texts_with_companies.append({
            "text":" ".join(splitted_text),
            "labels":labels
        })

In [38]:
df_final = pd.DataFrame(texts_with_companies)
df_final.to_csv("/home/musasina/projects/teknofest/msnet/datasets/train_final.csv",index=False)